In [ ]:
URL = "https://www.reddit.com/r/IllegallySmolCats"

https://requests.readthedocs.io/en/latest/

https://www.reddit.com/r/IllegallySmolCats/search/

https://www.reddit.com/wiki/search/

https://www.reddit.com/dev/api

https://www.reddit.com/dev/api/#GET_top



In [ ]:
from pathlib import Path
import requests
import shutil


https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent

In [ ]:
from platform import platform, python_version

user_agent = f"MyProject/0.1.0 (requests {requests.__version__}; Python {python_version()}; {platform()})"

print(user_agent)

In [ ]:
headers = {
    "User-Agent": user_agent,
}

# See listings in left panel
listing = "top"

params = {
    # "t": "all",
    "limit": "25",  # Capped at 100
}

response = requests.get(f"{URL}/{listing}.json", params=params, headers=headers)

In [ ]:
import json

content = json.loads(response.content)

In [ ]:
posts = content['data']['children']

assert len(posts) == content['data']['dist']

In [ ]:
for post in posts:
    print(post['data']['title'])

In [ ]:
from html import unescape

In [ ]:
def get_first_gallery_item_id(post):
    return post['data']['gallery_data']['items'][0]['media_id']

In [ ]:
def get_post_image_url(post):
    
    data = post['data']
    hint = data.get('post_hint')
    
    # Single image
    if hint == 'image':
        return data['url_overridden_by_dest']
        
    # Multiple images
    if gallery := data.get('gallery_data'):
        key = gallery['items'][0]['media_id']
        
        return data['media_metadata'][key]['p'][-1]['u']
    
    # Video
    if hint == 'hosted:video':
        return data['preview']['images'][0]['source']['url']

    # Cross-post
    return data['thumbnail']
    

    # Something else?
    raise RuntimeError(f"Unsupported post type for {data.get('url')}")
        

In [ ]:
get_post_image_url(posts[0])

In [ ]:
posts[0]

In [ ]:
# HTML rendering is completely optional
from IPython.display import display_html



styling = """
<style>
.container {
    display: grid;
    grid-template-columns: repeat(4, 1fr);
    gap: 40px 20px;
}
.data {
    grid-column: 1;
    display: flex;
    flex-direction: column;
    align-items: center;
    row-gap: 10px;
}
img {
    grid-column: 2/5;
    max-height: 400px;
    width: auto;
}
</style>
"""


def make_html_for_post(post):
    data = post['data']
    
    return '''
    <div class="data">
        <div><b>{score}</b> - {link_flair_text}</div>
        <div>u/{author}</div>
        <div>{title}</div>
    </div>
    <img src={img_src} />
    '''.format(**data, img_src = get_post_image_url(post))
    
    

rows = [make_html_for_post(post) for post in posts]


html = styling + '''<div class="container">''' + "".join(rows) + "</div>"

display_html(html, raw=True)